# Imports and Loads

In [1]:
from base import *
from scipy import stats as st
from services import *
from functools import reduce


/home/fabricio/.cache/pypoetry/virtualenvs/modelo-preditor-A4158a_0-py3.9/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
ATLANTICO_NORTE = AtlanticoNorte()
ATLANTICO_SUL = AtlanticoSul()
PACIFICO_SUL = PacificoSul()
NINO = Nino()
COORDENADAS = Coordenadas()
PRECIPITACAO = Precipitacao()
TIME = TimeOperations()
SELECTION = SelectionOperations()
PRECIPITACAO.encoder()

In [3]:
def gerar_base_com_deslomentos():
    base = {}
    for delta in range(1,13):
        col1 = []
        col2 = []
        col3 = []
        col4 = []
        for r in ATLANTICO_NORTE.data.columns:
            col1.append("TSM-AN"+r+"("+str(delta)+")")
        for r in ATLANTICO_SUL.data.columns:
            col2.append("TSM-AS"+r+"("+str(delta)+")")
        for r in PACIFICO_SUL.data.columns:
            col3.append("TSM-PS"+r+"("+str(delta)+")")
        for r in NINO.data.columns:
            col4.append("TSM-NI"+r+"("+str(delta)+")")
    
        a = ATLANTICO_NORTE.data.shift(delta)
        a.columns=col1
    
        b =ATLANTICO_SUL.data.shift(delta)
        b.columns=col2
    
        c=PACIFICO_SUL.data.shift(delta)
        c.columns=col3
    
        d=NINO.data.shift(delta)
        d.columns=col4
    
        data = [a,b,c,d]
        base[delta] = pd.concat(data,axis=1)
    return base

base = gerar_base_com_deslomentos()

### Calculando correlacoes

In [4]:
nino = {}
atln = {}
atls = {}
pcs = {}

areas = ["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11",
                  "PRM12","PRM13","PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21",
                  "PRM22","PRM23","PRM24"]

for delta in range(1,13):
    dc = {}
    for c in ATLANTICO_NORTE.data.columns:
        dc['TSM-AN'+c+"("+str(delta)+")"] = []
        var = []
        for p in PRECIPITACAO.data.columns:
            x = 'TSM-AN'+c+"("+str(delta)+")"
            y = 'PRM'+p
            a = pd.merge(ATLANTICO_NORTE.data[c].shift(delta),PRECIPITACAO.data[p], on=['Data'], how='outer')
            a.columns = [x,y]
            a = a.dropna()
            cor = st.kendalltau(a[x],a[y])[0]
            dc[x].append(cor)
        atln[delta] = dc


for delta in range(1,13):
    dc = {}
    for c in ATLANTICO_SUL.data.columns:
        dc['TSM-AS'+c+"("+str(delta)+")"] = []
        var = []
        for p in PRECIPITACAO.data.columns:
            x = 'TSM-AS'+c+"("+str(delta)+")"
            y = 'PRM'+p
            a = pd.merge(ATLANTICO_SUL.data[c].shift(delta),PRECIPITACAO.data[p], on=['Data'], how='outer')
            a.columns = [x,y]
            a = a.dropna()
            cor = st.kendalltau(a[x],a[y])[0]
            dc[x].append(cor)
        atls[delta] = dc

for delta in range(1,13):
    dc = {}
    for c in PACIFICO_SUL.data.columns:
        dc['TSM-AS'+c+"("+str(delta)+")"] = []
        var = []
        for p in PRECIPITACAO.data.columns:
            x = 'TSM-AS'+c+"("+str(delta)+")"
            y = 'PRM'+p
            a = pd.merge(PACIFICO_SUL.data[c].shift(delta),PRECIPITACAO.data[p], on=['Data'], how='outer')
            a.columns = [x,y]
            a = a.dropna()
            cor = st.kendalltau(a[x],a[y])[0]
            dc[x].append(cor)
        pcs[delta] = dc

for delta in range(1,13):
    dc = {}
    for c in NINO.data.columns:
        dc['TSM-NI'+c+"("+str(delta)+")"] = []
        var = []
        for p in PRECIPITACAO.data.columns:
            x = 'TSM-NI'+c+"("+str(delta)+")"
            y = 'PRM'+p
            a = pd.merge(NINO.data[c].shift(delta),PRECIPITACAO.data[p], on=['Data'], how='outer')
            a.columns = [x,y]
            a = a.dropna()
            cor = st.kendalltau(a[x],a[y])[0]
            dc[x].append(cor)
        nino[delta] = dc
        #dc['TSM-NI'+c+"("+str(delta)+")"].append(var)
#res = ad.DataFrame(dc).T
#res.columns = areas
#res.T

In [5]:
wr = pd.ExcelWriter('Correlacoes_Nino.xlsx')
conc1 = []
conc2 =[]
conc3 =[]
conc4 =[]

for k in atln.keys():
    conc1.append(pd.DataFrame(atln[k]))

for k in atls.keys():
    conc2.append(pd.DataFrame(atls[k]))

for k in pcs.keys():
    conc3.append(pd.DataFrame(pcs[k]))

for k in nino.keys():
    conc4.append(pd.DataFrame(nino[k]))
    

df = [pd.concat(conc1,axis=1), pd.concat(conc2,axis=1), pd.concat(conc3,axis=1), pd.concat(conc4,axis=1)]
df = pd.concat(df,axis=1)
df = df.set_index(pd.Index(areas))
df.to_excel("output/correlacoes/tabela_de_correlacoes.xlsx")

# SELECAO DE VARIAVEIS

In [6]:
selected = {}
for r in df.T.columns:
    data_abs = abs(df.T)
    indices = list(data_abs.sort_values(by=[str(r)], ascending=False)[r].head(7).index)
    selected[r] = indices

nvars = pd.DataFrame(selected)
nvars

,PRM1,PRM2,PRM3,PRM4,PRM5,PRM6,PRM7,PRM8,PRM9,PRM10,...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
0,TSM-NI3(1),TSM-NI3(4),TSM-NI3(4),TSM-NI3(4),TSM-NI3(4),TSM-NI3(4),TSM-NI3(4),TSM-NI3(4),TSM-NI3(4),TSM-NI3(4),...,TSM-NI3(4),TSM-NI3(4),TSM-NI3(4),TSM-NI3(4),TSM-NI3(11),TSM-NI3(11),TSM-NI3(6),TSM-NI3(4),TSM-NI3(4),TSM-NI3(4)
1,TSM-NI3(3),TSM-NI3(3),TSM-NI3(3),TSM-NI3(3),TSM-NI3(3),TSM-NI3(6),TSM-NI3(3),TSM-NI3(3),TSM-NI3(3),TSM-NI3(5),...,TSM-NI3(5),TSM-NI3(5),TSM-NI3(5),TSM-NI3(5),TSM-NI3(6),TSM-NI3(10),TSM-NI3(4),TSM-NI3(3),TSM-NI3(6),TSM-NI3(6)
2,TSM-NI3(4),TSM-NI3(5),TSM-NI3(5),TSM-NI3(5),TSM-NI3(5),TSM-NI3(5),TSM-NI3(1),TSM-NI3(5),TSM-NI3(5),TSM-NI3(3),...,TSM-NI3(3),TSM-NI3(6),TSM-NI3(3),TSM-NI3(6),TSM-NI3(12),TSM-NI3(6),TSM-NI3(5),TSM-NI3(5),TSM-NI3(5),TSM-NI3(5)
3,TSM-NI3(2),TSM-NI3(1),TSM-NI3(6),TSM-NI3(2),TSM-NI3(6),TSM-NI3(3),TSM-NI3(5),TSM-NI3(6),TSM-NI3(6),TSM-NI3(6),...,TSM-NI3(6),TSM-NI3(3),TSM-NI3(6),TSM-NI3(3),TSM-NI3(10),TSM-NI3(12),TSM-NI3(11),TSM-NI3(6),TSM-NI3(3),TSM-NI3(3)
4,TSM-NI3(5),TSM-NI3(2),TSM-NI3(2),TSM-NI3(6),TSM-NI3(2),TSM-NI3(7),TSM-NI3(2),TSM-NI3(2),TSM-NI3(2),TSM-NI3(1),...,TSM-NI3(2),TSM-NI4(6),TSM-NI3(1),TSM-NI3(1),TSM-NI3(7),TSM-NI3(4),TSM-NI3(10),TSM-NI3(2),TSM-NI4(6),TSM-NI3(1)
5,TSM-NI4(3),TSM-NI3(6),TSM-NI4(3),TSM-NI3(1),TSM-NI3(1),TSM-NI3(2),TSM-NI3(6),TSM-NI3(1),TSM-NI3(1),TSM-NI3(2),...,TSM-NI3(7),TSM-NI3(7),TSM-NI3(2),TSM-NI3(2),TSM-NI3(4),TSM-NI3(5),TSM-NI3(7),TSM-NI3(1),TSM-NI4(5),TSM-NI3(7)
6,TSM-NI3(6),TSM-NI4(3),TSM-NI3(1),TSM-NI4(5),TSM-NI3(7),TSM-NI3(8),TSM-NI4(3),TSM-NI4(3),TSM-NI4(3),TSM-NI4(3),...,TSM-NI3(1),TSM-NI4(5),TSM-NI3(7),TSM-NI3(7),TSM-NI3(5),TSM-NI3(9),TSM-NI3(12),TSM-NI4(6),TSM-NI3(1),TSM-NI3(2)


In [7]:
wr = pd.ExcelWriter('output/data/dados_defasados.xlsx')

for k in base.keys():
    base[k].to_excel(wr,sheet_name=str(k))

wr.save()

In [8]:
time = ['1','2','3','4','5','6','7','8','9','10','11','12']
sel = {}
for v in nvars.columns:
    sel[v] = []
    for r in nvars[v]:
        for t in time:
            read = pd.read_excel('output/data/dados_defasados.xlsx',t)
            if r in read.columns:
                sel[v].append(read[['Data',r]]) 
        #print()
    #print()
sel

{'PRM1': [          Data  TSM-NI3(1)
  0   1982-01-01         NaN
  1   1982-02-01       28.30
  2   1982-03-01       28.21
  3   1982-04-01       28.41
  4   1982-05-01       28.92
  ..         ...         ...
  463 2020-08-01       28.83
  464 2020-09-01       28.47
  465 2020-10-01       28.29
  466 2020-11-01       27.89
  467 2020-12-01       27.91
  
  [468 rows x 2 columns],
            Data  TSM-NI3(3)
  0   1982-01-01         NaN
  1   1982-02-01         NaN
  2   1982-03-01         NaN
  3   1982-04-01       28.30
  4   1982-05-01       28.21
  ..         ...         ...
  463 2020-08-01       28.94
  464 2020-09-01       29.07
  465 2020-10-01       28.83
  466 2020-11-01       28.47
  467 2020-12-01       28.29
  
  [468 rows x 2 columns],
            Data  TSM-NI3(4)
  0   1982-01-01         NaN
  1   1982-02-01         NaN
  2   1982-03-01         NaN
  3   1982-04-01         NaN
  4   1982-05-01       28.30
  ..         ...         ...
  463 2020-08-01       29.29
  464 

In [9]:
sel['PRM1']

[          Data  TSM-NI3(1)
 0   1982-01-01         NaN
 1   1982-02-01       28.30
 2   1982-03-01       28.21
 3   1982-04-01       28.41
 4   1982-05-01       28.92
 ..         ...         ...
 463 2020-08-01       28.83
 464 2020-09-01       28.47
 465 2020-10-01       28.29
 466 2020-11-01       27.89
 467 2020-12-01       27.91
 
 [468 rows x 2 columns],
           Data  TSM-NI3(3)
 0   1982-01-01         NaN
 1   1982-02-01         NaN
 2   1982-03-01         NaN
 3   1982-04-01       28.30
 4   1982-05-01       28.21
 ..         ...         ...
 463 2020-08-01       28.94
 464 2020-09-01       29.07
 465 2020-10-01       28.83
 466 2020-11-01       28.47
 467 2020-12-01       28.29
 
 [468 rows x 2 columns],
           Data  TSM-NI3(4)
 0   1982-01-01         NaN
 1   1982-02-01         NaN
 2   1982-03-01         NaN
 3   1982-04-01         NaN
 4   1982-05-01       28.30
 ..         ...         ...
 463 2020-08-01       29.29
 464 2020-09-01       28.94
 465 2020-10-01       

In [16]:
PRECIPITACAO.data.columns = ["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11","PRM12","PRM13",
                             "PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21","PRM22","PRM23","PRM24"]

data = []
prm = reduce(lambda left, right: pd.merge(left,right, on=['Data'], how='outer'),sel['PRM1'])
prm = pd.merge(prm,PRECIPITACAO.data['PRM1'], on=['Data'], how='outer')
data.append(prm)

prm1 = reduce(lambda left, right: pd.merge(left,right, on=['Data'], how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)'),data)
prm1.dropna().to_excel('prm1.xlsx')

In [20]:
PRECIPITACAO.data.columns = ["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11","PRM12","PRM13",
                             "PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21","PRM22","PRM23","PRM24"]
data = []
for k in sel.keys():
    prm = reduce(lambda left, right: pd.merge(left,right, on=['Data'], how='outer'),sel[k])
    prm = pd.merge(prm,PRECIPITACAO.data[k], on=['Data'], how='outer')
    data.append(prm)

In [21]:
base_tratada = reduce(lambda left, right: pd.merge(left,right, on=['Data'], how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)'),data)

X = base_tratada.drop(columns=["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11","PRM12","PRM13",
                             "PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21","PRM22","PRM23","PRM24"]).round(2)

y = base_tratada[["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11","PRM12","PRM13",
                             "PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21","PRM22","PRM23","PRM24"]]

base_tratada[list(X.columns) + list(y.columns)].to_excel("output/data/base_selecionada.xlsx")
base_tratada[list(X.columns) + list(y.columns)].dropna().to_excel("output/data/base_selecionada(dropna).xlsx")

In [316]:
X = X.round(2)
X

,Data,TSM-NI3(1),TSM-NI3(3),TSM-NI3(4),TSM-NI3(2),TSM-NI3(5),TSM-NI3(6),TSM-NI3(7),TSM-NI3(11),TSM-NI3(10),TSM-NI4(6),TSM-NI3(12)
0,1982-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1982-02-01,28.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1982-03-01,28.21,NaN,NaN,28.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1982-04-01,28.41,28.30,NaN,28.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1982-05-01,28.92,28.21,28.30,28.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
463,2020-08-01,28.83,28.94,29.29,29.07,29.22,29.17,29.28,29.33,29.64,27.14,29.50
464,2020-09-01,28.47,29.07,28.94,28.83,29.29,29.22,29.17,29.64,29.47,27.82,29.33
465,2020-10-01,28.29,28.83,29.07,28.47,28.94,29.29,29.22,29.47,29.50,28.32,29.64
466,2020-11-01,27.89,28.47,28.83,28.29,29.07,28.94,29.29,29.50,29.28,27.59,29.47


In [22]:
y

,PRM1,PRM2,PRM3,PRM4,PRM5,PRM6,PRM7,PRM8,PRM9,PRM10,...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
0,4,4,4,3,3,4,4,4,4,3,...,3,2,4,4,4,4,2,2,2,3
1,2,2,2,3,3,2,2,2,2,2,...,0,0,2,2,2,1,0,0,0,1
2,1,0,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,3,2,2,2,3,3,3,2,...,3,3,2,1,2,2,2,2,2,0
4,0,0,1,2,2,2,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,3,3,3,3,2,2,1,2,2,2,...,2,3,2,2,0,1,2,3,3,2
464,0,0,0,1,2,1,1,0,0,2,...,0,2,4,3,0,0,0,0,1,0
465,0,0,0,0,1,2,1,0,0,1,...,0,0,1,1,1,0,0,0,0,0
466,4,4,4,4,3,3,4,4,4,3,...,3,4,4,4,2,3,3,4,4,3


# Legacy

### Check Data Structures

In [4]:
def sumarize_data(data):
    print("Dimensions: {}   -  Rows: {}   Columns: {}".format(data.name, data.shape[0], data.shape[1]))
    print(str(r.name)+": NAN Values -  "+str(sum(r.isna().sum().values)))
    print()
    
datas = [ATLANTICO_NORTE.data, ATLANTICO_SUL.data, PACIFICO_SUL.data, NINO.data, COORDENADAS.data, PRECIPITACAO.data]
for r in datas:
    sumarize_data(r)

Dimensions: ATLANTICO_NORTE   -  Rows: 468   Columns: 20
ATLANTICO_NORTE: NAN Values -  0

Dimensions: ATLANTICO_SUL   -  Rows: 468   Columns: 40
ATLANTICO_SUL: NAN Values -  0

Dimensions: PACIFICO_SUL   -  Rows: 468   Columns: 35
PACIFICO_SUL: NAN Values -  0

Dimensions: NINO   -  Rows: 468   Columns: 5
NINO: NAN Values -  0

Dimensions: COORDENADAS   -  Rows: 24   Columns: 3
COORDENADAS: NAN Values -  0

Dimensions: PRECIPITACAO   -  Rows: 468   Columns: 25
PRECIPITACAO: NAN Values -  0



In [4]:
PRECIPITACAO.data['1'].value_counts()

NORMAL                    134
MUITO ABAIXO DO NORMAL    100
ABAIXO DO NORMAL           98
ACIMA DO NORMAL            86
MUITO ACIMA DO NORMAL      50
Name: 1, dtype: int64

### Execute Time and Structure Transformations

In [28]:
ATLANTICO_NORTE.transpose_to_time()
ATLANTICO_SUL.transpose_to_time()
PACIFICO_SUL.transpose_to_time()
NINO.transpose_to_time()
PRECIPITACAO.transpose_to_time()

In [35]:
PRECIPITACAO.data.head()

,Data,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
36,1982-01-01,MUITO ACIMA DO NORMAL,MUITO ACIMA DO NORMAL,MUITO ACIMA DO NORMAL,ACIMA DO NORMAL,ACIMA DO NORMAL,MUITO ACIMA DO NORMAL,MUITO ACIMA DO NORMAL,MUITO ACIMA DO NORMAL,MUITO ACIMA DO NORMAL,...,ACIMA DO NORMAL,NORMAL,MUITO ACIMA DO NORMAL,MUITO ACIMA DO NORMAL,MUITO ACIMA DO NORMAL,MUITO ACIMA DO NORMAL,NORMAL,NORMAL,NORMAL,ACIMA DO NORMAL
37,1982-02-01,NORMAL,NORMAL,NORMAL,ACIMA DO NORMAL,ACIMA DO NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,...,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,NORMAL,NORMAL,NORMAL,ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,ABAIXO DO NORMAL
38,1982-03-01,ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,ABAIXO DO NORMAL,ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,...,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL
39,1982-04-01,ACIMA DO NORMAL,ACIMA DO NORMAL,ACIMA DO NORMAL,NORMAL,NORMAL,NORMAL,ACIMA DO NORMAL,ACIMA DO NORMAL,ACIMA DO NORMAL,...,ACIMA DO NORMAL,ACIMA DO NORMAL,NORMAL,ABAIXO DO NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,MUITO ABAIXO DO NORMAL
40,1982-05-01,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,ABAIXO DO NORMAL,NORMAL,NORMAL,NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,ABAIXO DO NORMAL,...,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL,MUITO ABAIXO DO NORMAL


In [16]:
PRECIPITACAO.data.head()

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
Data,,,,,,,,,,,,,,,,,,,,,
1982-01-01,2,2,2,1,1,2,2,2,2,1,...,1,0,2,2,2,2,0,0,0,1
1982-02-01,0,0,0,1,1,0,0,0,0,0,...,-2,-2,0,0,0,-1,-2,-2,-2,-1
1982-03-01,-1,-2,-1,-2,-2,-1,-1,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
1982-04-01,1,1,1,0,0,0,1,1,1,0,...,1,1,0,-1,0,0,0,0,0,-2
1982-05-01,-2,-2,-1,0,0,0,-2,-2,-1,-2,...,-2,-2,-2,-2,-1,-2,-2,-2,-2,-2


In [49]:
atln = TIME.generate_displacement(ATLANTICO_NORTE.data, PRECIPITACAO.data, "ATLN")
atls = TIME.generate_displacement(ATLANTICO_SUL.data, PRECIPITACAO.data, "ATLS")
pcs = TIME.generate_displacement(PACIFICO_SUL.data, PRECIPITACAO.data, "PCS")
nino = TIME.generate_displacement(NINO.data, PRECIPITACAO.data, "NINO")

ValueError: Length mismatch: Expected axis has 468 elements, new values have 24 elements

In [19]:
def time_lag(x,y, delta,ocean_flag):
        flags = {"ATLN": "AN", "ATLS": "AS", "PCS": "PS", "NINO": "NI"}
        x = x[:len(x)-delta]
        y = y[delta:]
        lags = {}
        lags['Data'] = x['Data']
        for i in x.columns[1:]:
            lags["TSM-"+ocean+i+"("+str(delta)+")"] = x[i].values
        for r in y.columns[1:]:
            lags["PRM"+r] = y[r].values 
        return pd.DataFrame(lags)
    
def generate_displacement(data_in,data_out,ocean_flag):
        flags = {"ATLN": "AN", "ATLS": "AS", "PCS": "PS", "NINO": "NI"}
        out_columns = ["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11",
                  "PRM12","PRM13","PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21",
                  "PRM22","PRM23","PRM24"]
        DELTA = {}
        for r in range(1,13):
            x = data_in[r:]
            y = data_out[:len(data_out) - r]
            columns = []
            #columns.append(x.columns[0])
            for c in x.columns:
                columns.append("TSM-"+flags[ocean_flag]+str(c)+"("+str(r)+")")
            x.columns = columns
            y.columns = out_columns
            DELTA["DELTA "+str(r)] = {"X": x, "y": y}  
        return DELTA
    
atln = generate_displacement(ATLANTICO_NORTE.data, PRECIPITACAO.data, "ATLN")
atls = generate_displacement(ATLANTICO_SUL.data, PRECIPITACAO.data, "ATLS")
pcs = generate_displacement(PACIFICO_SUL.data, PRECIPITACAO.data, "PCS")
nino = generate_displacement(NINO.data, PRECIPITACAO.data, "NINO")

In [23]:
atln['DELTA 2']['X'].shape, atln['DELTA 2']['y'].shape
       
       

((466, 19), (466, 24))

In [103]:
wr = pd.ExcelWriter('output/Defasagens_Atlantico_Norte.xlsx')
wr2 = pd.ExcelWriter('output/Defasagens_Atlantico_Sul.xlsx')
wr3 = pd.ExcelWriter('output/Defasagens_Pacifico_Sul.xlsx')
wr4 = pd.ExcelWriter('output/Defasagens_Nino.xlsx')

for k in atln.keys():
    pd.DataFrame(atln[k]['X']).to_excel(wr, sheet_name=k)
    pd.DataFrame(atln[k]['y']).to_excel(wr, sheet_name="SAÍDAS-"+k)

for k in atls.keys():
    pd.DataFrame(atls[k]['X']).to_excel(wr2, sheet_name=k)
    pd.DataFrame(atls[k]['y']).to_excel(wr2, sheet_name="SAÍDAS-"+k)
    
for k in pcs.keys():
    pd.DataFrame(pcs[k]['X']).to_excel(wr3, sheet_name=k)
    pd.DataFrame(pcs[k]['y']).to_excel(wr3, sheet_name="SAÍDAS-"+k)

for k in nino.keys():
    pd.DataFrame(nino[k]['X']).to_excel(wr4, sheet_name=k)
    pd.DataFrame(nino[k]['y']).to_excel(wr4, sheet_name="SAÍDAS-"+k)

    
wr.save()
wr2.save()
wr3.save()
wr4.save()


#atln['DELTA 12']['X'].head()

In [21]:
ds = ATLANTICO_NORTE.data.copy() 
ds

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Data,,,,,,,,,,,,,,,,,,,
1982-01-01,26.0098,25.3862,24.3421,22.9730,26.8484,26.1652,25.2629,24.6777,26.8626,26.7427,26.4668,26.6040,27.7325,27.2840,27.3158,27.5030,28.2153,28.3471,28.4736
1982-02-01,25.4937,24.6246,23.7060,22.2753,26.4223,25.4757,24.6380,23.7625,26.7571,26.2851,26.1257,25.9726,26.0110,27.3053,27.2771,27.7486,28.5934,28.5578,28.8561
1982-03-01,25.3498,24.4863,23.5340,22.1519,26.3487,25.4243,24.6487,23.6527,26.7935,26.8084,26.4926,26.1522,26.1699,27.3521,27.5340,28.1318,28.8970,29.1426,29.1116
1982-04-01,25.7790,24.6510,23.4159,22.3043,26.6982,25.3215,24.6814,23.6110,27.1823,26.8794,26.5401,26.2872,27.1713,27.6002,27.6694,28.0068,28.8802,29.1436,28.6862
1982-05-01,26.6424,25.5152,24.1482,23.0335,27.3998,26.1700,25.3244,24.5587,27.5308,27.3391,26.9380,27.0607,27.8449,27.7344,27.6691,28.0665,28.1371,28.0639,27.7348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-01,28.4721,27.3896,26.5677,26.1410,28.9465,28.2160,27.9531,27.6894,29.2388,29.0570,28.2342,27.6240,27.0076,28.4813,27.8537,26.4444,25.3383,25.4426,26.1175
2020-09-01,28.6614,27.6689,27.1493,27.4002,29.1244,28.3055,28.1015,27.8288,28.9657,28.9848,28.6487,28.0024,27.3056,28.0232,27.8944,27.0353,26.2266,25.4192,26.0080
2020-10-01,28.4023,27.6131,27.3763,27.2015,28.9573,28.2666,27.9994,28.2323,29.1334,29.0081,28.8528,28.8224,28.3978,28.2930,28.4145,27.9301,27.1662,26.4760,26.7757


In [84]:
PRECIPITACAO.data[1:]

,Data,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
37,1982-02-01,0,0,0,1,1,0,0,0,0,...,-2,-2,0,0,0,-1,-2,-2,-2,-1
38,1982-03-01,-1,-2,-1,-2,-2,-1,-1,-2,-2,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
39,1982-04-01,1,1,1,0,0,0,1,1,1,...,1,1,0,-1,0,0,0,0,0,-2
40,1982-05-01,-2,-2,-1,0,0,0,-2,-2,-1,...,-2,-2,-2,-2,-1,-2,-2,-2,-2,-2
41,1982-06-01,-1,-1,-1,-1,0,1,0,0,0,...,-1,-1,0,0,0,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,2020-08-01,1,1,1,1,0,0,-1,0,0,...,0,1,0,0,-2,-1,0,1,1,0
500,2020-09-01,-2,-2,-2,-1,0,-1,-1,-2,-2,...,-2,0,2,1,-2,-2,-2,-2,-1,-2
501,2020-10-01,-2,-2,-2,-2,-1,0,-1,-2,-2,...,-2,-2,-1,-1,-1,-2,-2,-2,-2,-2
502,2020-11-01,2,2,2,2,1,1,2,2,2,...,1,2,2,2,0,1,1,2,2,1


### Execute Correlation Operations

In [27]:
 def generate_correlation_tables(data):
        CORR = {}
        data_list = []
        outs= ["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11",
               "PRM12","PRM13","PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21",
               "PRM22","PRM23","PRM24"]
        for k in data.keys():
            var = {}
            for x in data[k]['X'].columns:
                var[x] = []
                for y in data[k]['y'].columns:
                    c = st.kendalltau(data[k]['X'][x],data[k]['y'][y])[0]
                    #print(x,y, str(c))
                    #print(x+"("+str(len(data[k]['X'][x]))+")    "+y+"("+str(len(data[k]['y'][y]))+")")        
                    var[x].append(c)
            CORR[k] = pd.DataFrame(var)
            CORR[k]['AREAS'] = outs 
            CORR[k] = CORR[k].set_index('AREAS')
            #CORR[k].columns =  ["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11",
            #      "PRM12","PRM13","PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21",
            #      "PRM22","PRM23","PRM24"]
                #print()
        for k in CORR.keys():
            data_list.append(CORR[k])
        return reduce(lambda left, right: pd.merge(left,right, on=['AREAS'], how='outer'),data_list)
       
corr_atln = generate_correlation_tables(atln)
corr_atls = generate_correlation_tables(atls)
corr_pcs = generate_correlation_tables(pcs)
corr_nino = generate_correlation_tables(nino)

final_corr = pd.concat([corr_atln,corr_atls,corr_pcs,corr_nino],axis=1)
final_corr
final_corr.to_excel('output/tabela_de_correlacoes.xlsx')

In [31]:
final_corr = final_corr.T
final_corr



AREAS,PRM1,PRM2,PRM3,PRM4,PRM5,PRM6,PRM7,PRM8,PRM9,PRM10,...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
TSM-AN1(1),-0.048691,-0.059111,-0.062846,-0.043035,-0.034873,-0.024191,-0.067203,-0.075667,-0.070946,-0.044123,...,-0.059176,-0.036252,-0.021812,-0.025479,-0.074238,-0.075029,-0.077473,-0.036960,-0.027363,-0.049681
TSM-AN2(1),-0.044676,-0.057053,-0.059489,-0.039610,-0.031254,-0.020254,-0.066351,-0.069126,-0.066393,-0.039584,...,-0.059600,-0.039642,-0.019628,-0.027101,-0.078413,-0.074345,-0.070229,-0.040391,-0.035638,-0.055826
TSM-AN3(1),-0.037301,-0.044405,-0.044244,-0.026223,-0.013454,-0.003583,-0.059394,-0.058579,-0.050018,-0.020592,...,-0.049496,-0.022730,-0.001418,-0.017319,-0.079501,-0.074520,-0.063349,-0.033746,-0.028180,-0.043742
TSM-AN4(1),-0.028256,-0.031405,-0.029766,-0.013418,0.004616,0.014905,-0.048616,-0.048448,-0.037729,-0.009429,...,-0.035764,-0.007342,0.014317,-0.004106,-0.063875,-0.057812,-0.047421,-0.014494,-0.007397,-0.022768
TSM-AN5(1),-0.046232,-0.059360,-0.065072,-0.044809,-0.039686,-0.029748,-0.064648,-0.076300,-0.074296,-0.047352,...,-0.070689,-0.044661,-0.021450,-0.021217,-0.081895,-0.083502,-0.090629,-0.048499,-0.030218,-0.045701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSM-NI4(11),0.044006,0.040587,0.010113,0.003069,0.015250,0.005179,0.015996,0.032108,0.003837,-0.014453,...,-0.000823,0.016268,0.039529,0.030761,0.026993,0.017500,-0.002082,-0.000509,0.016241,0.034661
TSM-NI1(12),0.013315,0.006510,-0.008649,-0.036978,-0.039380,-0.040140,-0.013539,0.007460,-0.007214,-0.031638,...,-0.012393,-0.020886,-0.016725,-0.003631,-0.009787,0.007445,0.000305,-0.013999,-0.017269,-0.026437
TSM-NI2(12),0.038990,0.035232,0.001848,-0.019398,-0.012302,-0.017324,0.007561,0.029590,0.001535,-0.020471,...,-0.008580,-0.001774,0.017328,0.028859,0.009279,0.014776,-0.002798,-0.006311,0.001391,0.009740
TSM-NI3(12),0.026593,0.034409,0.013134,0.010331,0.014532,0.014680,-0.005563,-0.003501,-0.014982,-0.024682,...,-0.030027,-0.008171,0.022724,0.019807,-0.003289,-0.009611,-0.029799,-0.021133,0.004709,0.031312


In [32]:
final_corr

AREAS,PRM1,PRM2,PRM3,PRM4,PRM5,PRM6,PRM7,PRM8,PRM9,PRM10,...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
TSM-AN1(1),-0.048691,-0.059111,-0.062846,-0.043035,-0.034873,-0.024191,-0.067203,-0.075667,-0.070946,-0.044123,...,-0.059176,-0.036252,-0.021812,-0.025479,-0.074238,-0.075029,-0.077473,-0.036960,-0.027363,-0.049681
TSM-AN2(1),-0.044676,-0.057053,-0.059489,-0.039610,-0.031254,-0.020254,-0.066351,-0.069126,-0.066393,-0.039584,...,-0.059600,-0.039642,-0.019628,-0.027101,-0.078413,-0.074345,-0.070229,-0.040391,-0.035638,-0.055826
TSM-AN3(1),-0.037301,-0.044405,-0.044244,-0.026223,-0.013454,-0.003583,-0.059394,-0.058579,-0.050018,-0.020592,...,-0.049496,-0.022730,-0.001418,-0.017319,-0.079501,-0.074520,-0.063349,-0.033746,-0.028180,-0.043742
TSM-AN4(1),-0.028256,-0.031405,-0.029766,-0.013418,0.004616,0.014905,-0.048616,-0.048448,-0.037729,-0.009429,...,-0.035764,-0.007342,0.014317,-0.004106,-0.063875,-0.057812,-0.047421,-0.014494,-0.007397,-0.022768
TSM-AN5(1),-0.046232,-0.059360,-0.065072,-0.044809,-0.039686,-0.029748,-0.064648,-0.076300,-0.074296,-0.047352,...,-0.070689,-0.044661,-0.021450,-0.021217,-0.081895,-0.083502,-0.090629,-0.048499,-0.030218,-0.045701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSM-NI4(11),0.044006,0.040587,0.010113,0.003069,0.015250,0.005179,0.015996,0.032108,0.003837,-0.014453,...,-0.000823,0.016268,0.039529,0.030761,0.026993,0.017500,-0.002082,-0.000509,0.016241,0.034661
TSM-NI1(12),0.013315,0.006510,-0.008649,-0.036978,-0.039380,-0.040140,-0.013539,0.007460,-0.007214,-0.031638,...,-0.012393,-0.020886,-0.016725,-0.003631,-0.009787,0.007445,0.000305,-0.013999,-0.017269,-0.026437
TSM-NI2(12),0.038990,0.035232,0.001848,-0.019398,-0.012302,-0.017324,0.007561,0.029590,0.001535,-0.020471,...,-0.008580,-0.001774,0.017328,0.028859,0.009279,0.014776,-0.002798,-0.006311,0.001391,0.009740
TSM-NI3(12),0.026593,0.034409,0.013134,0.010331,0.014532,0.014680,-0.005563,-0.003501,-0.014982,-0.024682,...,-0.030027,-0.008171,0.022724,0.019807,-0.003289,-0.009611,-0.029799,-0.021133,0.004709,0.031312


In [37]:
final_corr

AREAS,PRM1,PRM2,PRM3,PRM4,PRM5,PRM6,PRM7,PRM8,PRM9,PRM10,...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
TSM-AN1(1),-0.048691,-0.059111,-0.062846,-0.043035,-0.034873,-0.024191,-0.067203,-0.075667,-0.070946,-0.044123,...,-0.059176,-0.036252,-0.021812,-0.025479,-0.074238,-0.075029,-0.077473,-0.036960,-0.027363,-0.049681
TSM-AN2(1),-0.044676,-0.057053,-0.059489,-0.039610,-0.031254,-0.020254,-0.066351,-0.069126,-0.066393,-0.039584,...,-0.059600,-0.039642,-0.019628,-0.027101,-0.078413,-0.074345,-0.070229,-0.040391,-0.035638,-0.055826
TSM-AN3(1),-0.037301,-0.044405,-0.044244,-0.026223,-0.013454,-0.003583,-0.059394,-0.058579,-0.050018,-0.020592,...,-0.049496,-0.022730,-0.001418,-0.017319,-0.079501,-0.074520,-0.063349,-0.033746,-0.028180,-0.043742
TSM-AN4(1),-0.028256,-0.031405,-0.029766,-0.013418,0.004616,0.014905,-0.048616,-0.048448,-0.037729,-0.009429,...,-0.035764,-0.007342,0.014317,-0.004106,-0.063875,-0.057812,-0.047421,-0.014494,-0.007397,-0.022768
TSM-AN5(1),-0.046232,-0.059360,-0.065072,-0.044809,-0.039686,-0.029748,-0.064648,-0.076300,-0.074296,-0.047352,...,-0.070689,-0.044661,-0.021450,-0.021217,-0.081895,-0.083502,-0.090629,-0.048499,-0.030218,-0.045701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSM-NI4(11),0.044006,0.040587,0.010113,0.003069,0.015250,0.005179,0.015996,0.032108,0.003837,-0.014453,...,-0.000823,0.016268,0.039529,0.030761,0.026993,0.017500,-0.002082,-0.000509,0.016241,0.034661
TSM-NI1(12),0.013315,0.006510,-0.008649,-0.036978,-0.039380,-0.040140,-0.013539,0.007460,-0.007214,-0.031638,...,-0.012393,-0.020886,-0.016725,-0.003631,-0.009787,0.007445,0.000305,-0.013999,-0.017269,-0.026437
TSM-NI2(12),0.038990,0.035232,0.001848,-0.019398,-0.012302,-0.017324,0.007561,0.029590,0.001535,-0.020471,...,-0.008580,-0.001774,0.017328,0.028859,0.009279,0.014776,-0.002798,-0.006311,0.001391,0.009740
TSM-NI3(12),0.026593,0.034409,0.013134,0.010331,0.014532,0.014680,-0.005563,-0.003501,-0.014982,-0.024682,...,-0.030027,-0.008171,0.022724,0.019807,-0.003289,-0.009611,-0.029799,-0.021133,0.004709,0.031312


In [29]:
'''
lists = []
wr = pd.ExcelWriter('Base_Atlantico_Norte.xlsx')
wr2 = pd.ExcelWriter('Base_Atlantico_Sul.xlsx')
wr3 = pd.ExcelWriter('Base_Pacifico_Sul.xlsx')
wr4 = pd.ExcelWriter('Base_Nino.xlsx')

for d in range(1,13):
    TIME.time_lag(ATLANTICO_NORTE.data, PRECIPITACAO.data, d,'AN').to_excel(wr, sheet_name=str(d))
    TIME.time_lag(ATLANTICO_SUL.data, PRECIPITACAO.data, d,'AS').to_excel(wr2, sheet_name=str(d))
    TIME.time_lag(PACIFICO_SUL.data, PRECIPITACAO.data, d,'PS').to_excel(wr3, sheet_name=str(d))
    TIME.time_lag(NINO.data, PRECIPITACAO.data, d,'NI').to_excel(wr4, sheet_name=str(d))
wr.save()
wr2.save()
wr3.save()
wr4.save()

'''

In [50]:



final_corr

AREAS,PRM1,PRM2,PRM3,PRM4,PRM5,PRM6,PRM7,PRM8,PRM9,PRM10,...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
TSM-AN1(1),-0.048691,-0.059111,-0.062846,-0.043035,-0.034873,-0.024191,-0.067203,-0.075667,-0.070946,-0.044123,...,-0.059176,-0.036252,-0.021812,-0.025479,-0.074238,-0.075029,-0.077473,-0.036960,-0.027363,-0.049681
TSM-AN2(1),-0.044676,-0.057053,-0.059489,-0.039610,-0.031254,-0.020254,-0.066351,-0.069126,-0.066393,-0.039584,...,-0.059600,-0.039642,-0.019628,-0.027101,-0.078413,-0.074345,-0.070229,-0.040391,-0.035638,-0.055826
TSM-AN3(1),-0.037301,-0.044405,-0.044244,-0.026223,-0.013454,-0.003583,-0.059394,-0.058579,-0.050018,-0.020592,...,-0.049496,-0.022730,-0.001418,-0.017319,-0.079501,-0.074520,-0.063349,-0.033746,-0.028180,-0.043742
TSM-AN4(1),-0.028256,-0.031405,-0.029766,-0.013418,0.004616,0.014905,-0.048616,-0.048448,-0.037729,-0.009429,...,-0.035764,-0.007342,0.014317,-0.004106,-0.063875,-0.057812,-0.047421,-0.014494,-0.007397,-0.022768
TSM-AN5(1),-0.046232,-0.059360,-0.065072,-0.044809,-0.039686,-0.029748,-0.064648,-0.076300,-0.074296,-0.047352,...,-0.070689,-0.044661,-0.021450,-0.021217,-0.081895,-0.083502,-0.090629,-0.048499,-0.030218,-0.045701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSM-NI4(11),0.044006,0.040587,0.010113,0.003069,0.015250,0.005179,0.015996,0.032108,0.003837,-0.014453,...,-0.000823,0.016268,0.039529,0.030761,0.026993,0.017500,-0.002082,-0.000509,0.016241,0.034661
TSM-NI1(12),0.013315,0.006510,-0.008649,-0.036978,-0.039380,-0.040140,-0.013539,0.007460,-0.007214,-0.031638,...,-0.012393,-0.020886,-0.016725,-0.003631,-0.009787,0.007445,0.000305,-0.013999,-0.017269,-0.026437
TSM-NI2(12),0.038990,0.035232,0.001848,-0.019398,-0.012302,-0.017324,0.007561,0.029590,0.001535,-0.020471,...,-0.008580,-0.001774,0.017328,0.028859,0.009279,0.014776,-0.002798,-0.006311,0.001391,0.009740
TSM-NI3(12),0.026593,0.034409,0.013134,0.010331,0.014532,0.014680,-0.005563,-0.003501,-0.014982,-0.024682,...,-0.030027,-0.008171,0.022724,0.019807,-0.003289,-0.009611,-0.029799,-0.021133,0.004709,0.031312


In [65]:
def select_nvars(data, n):
        data = data
        columns = data.columns
        selected = {}
        for r in data.columns:
            data_abs = abs(data)
            indices = list(data_abs.sort_values(by=[str(r)], ascending=False)[r].head(n).index)
            selected[r] = indices
        return pd.DataFrame(selected)
    
def read_multiple_bases():
        bases = {}
        paths = ["Base_Atlantico_Norte.xlsx","Base_Atlantico_Sul.xlsx", "Base_Nino.xlsx", "Base_Pacifico_Sul.xlsx"]
        sheet_names = ['1','2','3','4','5','6','7','8','9','10','11','12']
        i = 0
        for p in paths:
            for s in sheet_names:
                bases[i+1] = pd.read_excel(p, s)
                i+=1
        return bases
    
def find_var(var, out, data):
        col = list(data.columns)
        if var in col: 
            return data[[var,out]]
        return None
    
    
def select_data(data, n):
        selected_vars = select_nvars(data, n)
        var = {}
        vars_name = {}
        bases = read_multiple_bases()
        for v in selected_vars.columns:
            var[v] = []
            for i in selected_vars[v]:
                for k in bases.keys():
                    df = find_var(i,v,bases[k])
                    if df is not None:
                        var[v].append(df)
        for k in var.keys():
            vars_name[k] = []
            for i in range(0,len(var[k])):
                vars_name[k].append(var[k][i].columns[0])
        return vars_name, var


vars_name, var = select_data(final_corr,5)
vars_name = pd.DataFrame(vars_name)
vars_name
#selected = select_nvars(final_corr, 5)
#selected

,PRM1,PRM2,PRM3,PRM4,PRM5,PRM6,PRM7,PRM8,PRM9,PRM10,...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
0,TSM-NI3(1),TSM-NI3(1),TSM-NI3(1),TSM-NI3(1),TSM-NI3(2),TSM-NI3(2),TSM-AN15(4),TSM-NI3(1),TSM-NI3(1),TSM-NI3(1),...,TSM-AN14(9),TSM-AN15(8),TSM-NI3(1),TSM-NI3(2),TSM-AN15(3),TSM-AN14(2),TSM-AN15(8),TSM-NI3(1),TSM-AN16(5),TSM-NI3(1)
1,TSM-NI3(2),TSM-NI3(2),TSM-NI4(1),TSM-NI4(1),TSM-NI3(1),TSM-NI3(1),TSM-NI3(1),TSM-AN14(9),TSM-AN15(8),TSM-NI3(2),...,TSM-AN15(8),TSM-AN14(8),TSM-NI3(2),TSM-NI3(1),TSM-AN14(2),TSM-AN15(1),TSM-AN14(2),TSM-NI3(2),TSM-AN15(8),TSM-NI3(2)
2,TSM-NI4(1),TSM-NI4(1),TSM-NI3(2),TSM-NI3(2),TSM-NI4(1),TSM-NI2(1),TSM-AN15(3),TSM-AN15(8),TSM-NI3(2),TSM-NI4(1),...,TSM-AN14(8),TSM-NI3(2),TSM-AN15(8),TSM-NI4(3),TSM-AN15(1),TSM-AN15(8),TSM-AN14(9),TSM-AS34(3),TSM-NI3(1),TSM-NI4(1)
3,TSM-NI3(3),TSM-NI3(3),TSM-NI2(1),TSM-NI2(1),TSM-NI2(1),TSM-NI2(2),TSM-AN14(4),TSM-AN15(9),TSM-AN14(9),TSM-AN15(6),...,TSM-AN15(9),TSM-NI3(1),TSM-PS1(2),TSM-NI3(3),TSM-AN15(2),TSM-AN15(4),TSM-AN14(8),TSM-AN15(8),TSM-PS4(5),TSM-AN16(5)
4,TSM-NI4(2),TSM-NI4(2),TSM-AN15(8),TSM-NI4(2),TSM-NI4(2),TSM-NI2(3),TSM-NI3(2),TSM-NI3(2),TSM-AN15(9),TSM-NI3(3),...,TSM-NI3(1),TSM-AN15(6),TSM-NI4(1),TSM-NI4(1),TSM-AN15(4),TSM-AN15(2),TSM-AN15(1),TSM-AN14(8),TSM-NI1(4),TSM-AN15(8)


In [72]:
paths = {"AN": "output/Defasagens_Atlantico_Norte.xlsx", "AS": "output/Defasagens_Atlantico_Sul.xlsx", 
         "PS": "output/Defasagens_Pacifico_Sul.xlsx","NI": "output/Defasagens_Nino.xlsx"}

sheet_names = ['DELTA 1','DELTA 2','DELTA 3','DELTA 4','DELTA 5','DELTA 6', 
               'DELTA 7','DELTA 8','DELTA 9','DELTA 10','DELTA 11','DELTA 12']
bases = {}
variaveis = {}
i=0
for p in paths.keys():
    bases[p] = []
    for s in sheet_names:
        bases[p].append(pd.read_excel(paths[p],s))
        i+=1
        
'''
for col in vars_name.columns:
    variaveis[col] = []
    for item in vars_name[col]:
        for i in bases:
            if item in bases[i].columns:
                variaveis[col].append(pd.DataFrame({"Data": bases[i][bases[i].columns[0]], str(item): bases[i][item].values}))
'''        



'\nfor col in vars_name.columns:\n    variaveis[col] = []\n    for item in vars_name[col]:\n        for i in bases:\n            if item in bases[i].columns:\n                variaveis[col].append(pd.DataFrame({"Data": bases[i][bases[i].columns[0]], str(item): bases[i][item].values}))\n'

In [111]:



data_selected = {}
found = None

for c in vars_name.columns:
    data_selected[c] = []
    for row in vars_name[c]:
        for r in bases.keys():
            for df in bases[r]:
                #print(df.columns)
                if row in df.columns:
                    #print(True)
                    data_selected[c].append(df[['Data',row]])
    #print()
                    
#data_selected['PRM1']


#bases['NI'][2][['Data','TSM-NI3(3)']]

In [117]:
#pd.concat(data_selected['PRM1'], on='Data', join='outer')
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Data'],
                                            how='outer'), data_selected['PRM1'])

df_merged

,Data,TSM-NI3(1),TSM-NI3(2),TSM-NI4(1),TSM-NI3(3),TSM-NI4(2)
0,1982-02-01,28.21,NaN,26.70,NaN,NaN
1,1982-03-01,28.41,28.41,27.20,NaN,27.20
2,1982-04-01,28.92,28.92,28.02,28.92,28.02
3,1982-05-01,29.49,29.49,28.54,29.49,28.54
4,1982-06-01,29.76,29.76,28.75,29.76,28.75
...,...,...,...,...,...,...
462,2020-08-01,28.47,28.47,26.18,28.47,26.18
463,2020-09-01,28.29,28.29,25.77,28.29,25.77
464,2020-10-01,27.89,27.89,25.30,27.89,25.30
465,2020-11-01,27.91,27.91,25.34,27.91,25.34


### Célula Anterior

TypeError: unhashable type: 'dict_keys'

In [175]:
#merged_nan =reduce(lambda  left,right: pd.concat(left,right,join='inner'), out['PRM1'])
out_keys = []
for r in out.keys(): 
    out_keys.append(out[r])
    
merged_nan = reduce(lambda left, right: pd.merge(left,right, how="outer"), out_keys)
#merged_nan = merged_nan.drop_duplicates()
tmp1 = merged_nan.drop(columns=["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11",
               "PRM12","PRM13","PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21",
               "PRM22","PRM23","PRM24"])
tmp2 = merged_nan[["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11",
               "PRM12","PRM13","PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21",
               "PRM22","PRM23","PRM24"]]
#merged_nan = pd.concat([tmp1,tmp2])

merged_nan = merged_nan[['Data','TSM-NI3(1)','TSM-NI3(2)','TSM-NI4(1)','TSM-NI3(3)','TSM-NI4(2)','TSM-NI2(1)','TSM-AN15(8)','TSM-NI2(2)','TSM-NI2(3)','TSM-AN15(4)',
 'TSM-AN15(3)','TSM-AN14(4)','TSM-AN14(9)','TSM-AN15(9)','TSM-AN15(6)','TSM-AN14(6)','TSM-AN14(8)','TSM-AN14(2)','TSM-AN15(1)','TSM-AN15(2)',
 'TSM-PS1(2)','TSM-NI4(3)','TSM-AS34(3)','TSM-AN16(5)','TSM-PS4(5)','TSM-NI1(4)','PRM1','PRM2','PRM3','PRM4','PRM5','PRM6','PRM7','PRM8','PRM9',
 'PRM10','PRM11','PRM12','PRM13','PRM14','PRM15','PRM16','PRM17','PRM18','PRM19','PRM20','PRM21','PRM22','PRM23','PRM24']]

merged_nan

,Data,TSM-NI3(1),TSM-NI3(2),TSM-NI4(1),TSM-NI3(3),TSM-NI4(2),TSM-NI2(1),TSM-AN15(8),TSM-NI2(2),TSM-NI2(3),...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
0,1982-02-01,28.21,NaN,26.70,NaN,NaN,26.38,NaN,NaN,NaN,...,-2,-2,0,0,0,-1,-2,-2,-2,-1
1,1982-03-01,28.41,28.41,27.20,NaN,27.20,26.98,NaN,26.98,NaN,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
2,1982-04-01,28.92,28.92,28.02,28.92,28.02,27.68,NaN,27.68,27.68,...,1,1,0,-1,0,0,0,0,0,-2
3,1982-05-01,29.49,29.49,28.54,29.49,28.54,27.79,NaN,27.79,27.79,...,-2,-2,-2,-2,-1,-2,-2,-2,-2,-2
4,1982-06-01,29.76,29.76,28.75,29.76,28.75,27.46,NaN,27.46,27.46,...,-1,-1,0,0,0,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,2020-08-01,28.47,28.47,26.18,28.47,26.18,24.42,27.8537,24.42,24.42,...,0,1,0,0,-2,-1,0,1,1,0
463,2020-09-01,28.29,28.29,25.77,28.29,25.77,23.58,27.8944,23.58,23.58,...,-2,0,2,1,-2,-2,-2,-2,-1,-2
464,2020-10-01,27.89,27.89,25.30,27.89,25.30,23.61,28.4145,23.61,23.61,...,-2,-2,-1,-1,-1,-2,-2,-2,-2,-2
465,2020-11-01,27.91,27.91,25.34,27.91,25.34,23.82,28.4594,23.82,23.82,...,1,2,2,2,0,1,1,2,2,1


In [179]:
merged = merged_nan.dropna()
merged_nan.to_excel("submission_data.xlsx")
merged.to_excel("submission_data(dropna).xlsx")

In [134]:
def change_col_names(cols, y):
    new_cols = ['Data']
    for r in range(1,len(cols)-1):
        new_cols.append('X_'+y+"_"+str(r))
    new_cols.append(y)
    return new_cols

dfs = []
for k in out.keys():
    #out[k].columns = change_col_names(out[k].columns, k)
    dfs.append(out[k])
    
merged = reduce(lambda  left,right: pd.merge(left,right,on=['Data'],suffixes=('','_DROP'),how='outer').filter(regex='^(?!.*_DROP)'), dfs).dropna()

merged.to_excel("model_data_submission(dropna).xlsx")

In [133]:
merged.columns

Index(['Data', 'TSM-NI3(1)', 'TSM-NI3(2)', 'TSM-NI4(1)', 'TSM-NI3(3)',
       'TSM-NI4(2)', 'PRM1', 'TSM-NI2(1)', 'TSM-AN15(8)', 'TSM-NI2(2)',
       'TSM-NI2(3)', 'TSM-AN15(4)', 'TSM-AN15(3)', 'TSM-AN14(4)',
       'TSM-AN14(9)', 'TSM-AN15(9)', 'TSM-AN15(6)', 'TSM-AN14(6)',
       'TSM-AN14(8)', 'TSM-AN14(2)', 'TSM-AN15(1)', 'TSM-AN15(2)',
       'TSM-PS1(2)', 'TSM-NI4(3)', 'TSM-AS34(3)', 'TSM-AN16(5)', 'TSM-PS4(5)',
       'TSM-NI1(4)'],
      dtype='object')

In [26]:
NINO.data.T[1:]

,1,2,3,4
1982-02-01,25.49,26.38,28.21,26.7
1982-03-01,25.21,26.98,28.41,27.2
1982-04-01,24.5,27.68,28.92,28.02
1982-05-01,23.97,27.79,29.49,28.54
1982-06-01,22.89,27.46,29.76,28.75
...,...,...,...,...
2020-08-01,19.69,24.42,28.47,26.18
2020-09-01,19.48,23.58,28.29,25.77
2020-10-01,19.67,23.61,27.89,25.3
2020-11-01,20.94,23.82,27.91,25.34


In [100]:
writer = pd.ExcelWriter('model_data_submission.xlsx')
dfs = []
for k in out.keys():
    dfs.append(pd.DataFrame(out[k]))
    out[k].to_excel(writer, sheet_name=k)



writer.save()

In [73]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Data'],
                                            how='outer'), variaveis['PRM1'])
df_merged = pd.merge(df_merged, tmp[['Data', '1']], on='Data')
df_merged.to_excel("teste.xlsx")

# Modelling

In [17]:
data = pd.read_excel('submission_data(dropna).xlsx')
data = data.head()
#data = data.drop('Unnamed: 0')
X = data[data.columns[3:28]]
y = data[data.columns[28:]]

In [18]:
X

,TSM-NI3(2),TSM-NI4(1),TSM-NI3(3),TSM-NI4(2),TSM-NI2(1),TSM-AN15(8),TSM-NI2(2),TSM-NI2(3),TSM-AN15(4),TSM-AN15(3),...,TSM-AN14(8),TSM-AN14(2),TSM-AN15(1),TSM-AN15(2),TSM-PS1(2),TSM-NI4(3),TSM-AS34(3),TSM-AN16(5),TSM-PS4(5),TSM-NI1(4)
0,29.38,28.64,29.38,28.64,27.11,27.1835,27.11,27.11,27.1835,27.1835,...,27.1596,27.1596,27.1835,27.1835,27.4690,28.64,18.7258,26.9433,24.6677,22.94
1,29.23,28.81,29.23,28.81,27.62,27.3207,27.62,27.62,27.3207,27.3207,...,27.3285,27.3285,27.3207,27.3207,28.0856,28.81,18.9325,27.0347,25.1475,24.59
2,29.15,29.21,29.15,29.21,28.39,27.1600,28.39,28.39,27.1600,27.1600,...,27.3906,27.3906,27.1600,27.1600,28.9323,29.21,20.5936,26.7475,26.2033,26.13
3,29.00,29.36,29.00,29.36,28.92,27.1572,28.92,28.92,27.1572,27.1572,...,27.5280,27.5280,27.1572,27.1572,29.7356,29.36,22.9802,27.1061,26.7675,27.42
4,28.79,29.13,28.79,29.13,28.92,27.3597,28.92,28.92,27.3597,27.3597,...,27.6645,27.6645,27.3597,27.3597,30.2312,29.13,23.8416,27.5774,27.4832,28.09


In [19]:
y

,PRM1,PRM2,PRM3,PRM4,PRM5,PRM6,PRM7,PRM8,PRM9,PRM10,...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
0,1,1,0,0,0,0,0,-1,-1,-1,...,-1,-1,-1,-1,0,-1,-2,-2,-2,-1
1,-2,-2,-2,-2,-2,-2,-2,-2,-1,-1,...,-1,0,0,-2,1,0,0,0,0,-1
2,0,0,-2,-2,-2,-1,0,-1,-1,-1,...,0,-1,0,-1,-1,0,0,0,0,0
3,-2,-2,-1,-1,-1,-1,-1,-1,0,0,...,0,0,0,1,-1,-1,0,0,1,2
4,0,0,-1,-2,-2,-1,0,0,-1,-1,...,-1,-1,0,0,0,0,-1,-2,-1,0


In [22]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
'''
cls = autosklearn.classification.AutoSklearnClassifier(
    metric=autosklearn.metrics.roc_auc,
    n_jobs=-1,
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds': 5},
)
'''
cls = MultiOutputClassifier(forest, n_jobs=-1)

NameError: name 'forest' is not defined

In [21]:
cls.fit(X,y)

ValueError: Classification with data of type multiclass-multioutput is not supported. Supported types are ['binary', 'multiclass', 'multilabel-indicator']. You can find more information about scikit-learn data types in: https://scikit-learn.org/stable/modules/multiclass.html